# Karimi Proposal

In [1]:
#DO NOT CHANGE THIS CODE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pmtools2 as pm
import kmodes
%matplotlib inline

In [2]:
# LOAD DATA
#DO NOT CHANGE THIS CODE
df_emp_access = pd.read_csv('train.csv')
attnames_list = list(df_emp_access.columns)
print(attnames_list)
#A_III Statistics

#DO NOT CHANGE THIS CODE
entries = df_emp_access.values.tolist()
attidx_to_collcounter = pm.get_entries_freqs(entries)

print('#Entries:', len(entries))
print('#Users:', len(df_emp_access[attnames_list[2:]].drop_duplicates()))
print('#Resources:', len(df_emp_access.RESOURCE.drop_duplicates()))
print()

print('---Attributes---')
for att_idx in range(len(attnames_list)):
    print('Num values', attnames_list[att_idx], ':', len(attidx_to_collcounter[att_idx].keys()))
num_resources = attnames_list[1]
    
print()
df_pos_entries = df_emp_access[df_emp_access['ACTION']==1]
df_neg_entries = df_emp_access[df_emp_access['ACTION']==0]
print('Num positive entries:',len(df_pos_entries))
print('Num negative entries:',len(df_neg_entries))

df_emp_access

['ACTION', 'RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']
#Entries: 32769
#Users: 9561
#Resources: 7518

---Attributes---
Num values ACTION : 2
Num values RESOURCE : 7518
Num values MGR_ID : 4243
Num values ROLE_ROLLUP_1 : 128
Num values ROLE_ROLLUP_2 : 177
Num values ROLE_DEPTNAME : 449
Num values ROLE_TITLE : 343
Num values ROLE_FAMILY_DESC : 2358
Num values ROLE_FAMILY : 67
Num values ROLE_CODE : 343

Num positive entries: 30872
Num negative entries: 1897


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325
...,...,...,...,...,...,...,...,...,...,...
32764,1,23497,16971,117961,118300,119993,118321,240983,290919,118322
32765,1,25139,311198,91261,118026,122392,121143,173805,249618,121145
32766,1,34924,28805,117961,118327,120299,124922,152038,118612,124924
32767,1,80574,55643,118256,118257,117945,280788,280788,292795,119082


## Data pre-processing
1. Continouos to Categorical Values
2. Handle missing values - New value

In [3]:
# df_data = df_pos_entries[attnames_list[2:]].drop_duplicates()
# print('Drop duplicates:', len(df_data))

# pos_entries = df_data.values.tolist()
pos_entries = df_pos_entries.values.tolist()
neg_entries = df_neg_entries.values.tolist()

## Selection of Learning Algorithm
1. K-modes algorithm

In [4]:
###Select the number of clusters###
num_clusters = 15

#DO NOT CHANGE THIS CODE
# seed = 29

#Compute centroids and labels
# num_init = 5
centroids = []
kmodes_huang = kmodes.KModes(n_clusters=num_clusters, init='Huang', verbose=0)
cluster_labels = kmodes_huang.fit_predict(df_pos_entries)
centroids = kmodes_huang.cluster_centroids_

print('Ready!')    

Ready!


In [5]:
df_pos_entries["cls"] = cluster_labels

/home/daniel/anaconda3/envs/tesis_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Parameter tuning
1. Number of clusters (Silhouette Method)
2. Cluster initialization

## Policy Rules Extraction

In [8]:
def freq(value, attribute, dataplace):
    """
    Calculate the frequency of the value in the dataplace.

    Parameters
    ----------
    value : int
        Value to compute its frequency.

    attribute : string
        Name of the attribute.

    dataplace : DataFrame pandas
        Data to search.
    
    Returns
    -------
    float [0-1]
        Returns the value of frequency of the value in the data.
    """
    value_freq = dataplace[dataplace[attribute] == value].drop_duplicates()
    return len(value_freq) / len(dataplace)

def freq_rels(attrA, attrB, dataplace):
    """
    Compute the frequency of the attribute relation.

    Parameters
    ----------
    attrA : string
        Name of the attribute A to compare.
    attrB : string
        Name of the attribute B to compare.
    dataplace : DataFrame pandas
        Data to search.

    Returns
    -------
    float [0-1]
        Returns the value of frequency of the range of values in the data.
    """
    # Get the range of values of attribute A.
    range_val_A = set(dataplace[attrA].values.tolist())

    # Get the range of values of attribute B.
    range_val_B = set(dataplace[attrB].values.tolist())

    # Check if the len
    if len(range_val_A) == len(range_val_B):
        # Compute the intersection
        inter_A_B = range_val_A.intersection(range_val_B)
        if len(inter_A_B) == len(range_val_A):
            boolean_series = dataplace[attrA].isin(inter_A_B)
            frac_log = dataplace[boolean_series]
            return len(frac_log) / len(dataplace) # Return the fraction
        return 0
    else:
        return 0    

def extract_attributes_filters(C_i, A, L, posThr, negThr):
    """
    Effective attribute extraction algorithm. Generate a rule for each cluster.

    Parameters
    ----------
    C_i : DataFrame pandas
        Access request in the Cluster i.

    A : List
        List of attributes.

    V : List
        Values of attributes.

    L : DataFrame
        Complete Access Log.

    PosThr : float
        Positive Threshold to the effective positive attribute.

    NegThr : float
        Negative Threshold to the effective negative attribute.

    Returns
    -------
    list
        Returns the rule with the effective attributes for the cluster i.
    """
    filter_to_ret = [] # Rule
    for a in A:        
        a_values = C_i[a].drop_duplicates().tolist()        
        for v in a_values:
            if freq(v, a, C_i) - freq(v, a, L) > posThr:
                if not [a, v] in filter_to_ret:
                    filter_to_ret.append([a, v])
            if freq(v, a, L) - freq(v, a, C_i) > negThr:
                if not [a, -1*v] in filter_to_ret:
                    filter_to_ret.append([a, v*-1])
    return filter_to_ret

def extract_relations(C_i, A, L, posThr, negThr):
    """
    Extract the effective relation. For each cluster.

    Parameters
    ----------
    C_i : DataFrame pandas
        Access request in the Cluster i.

    A : List
        List of attributes.

    L : DataFrame
        Complete Access Log.

    posThr : float
        Positive Threshold to the effective positive relation.

    negThr : float
        Negative Threshold to the effective negative relation.

    Returns
    -------
    list
        Returns the rule with the effective relation for the cluster i.
    """
    relation_to_ret = []
    for a in A:
        for b in A:
            if a != b:
                if freq_rels(a, b, C_i) - freq_rels(a, b, L) > posThr:
                    if not [a, b] in relation_to_ret:
                        relation_to_ret.append([a, b])
                if freq_rels(a, b, L) - freq_rels(a, b, C_i) > negThr:
                    if not [a, '!'+b] in relation_to_ret:                        
                        relation_to_ret.append([a, '!'+b])
                    #print()

def rule_inference(data_, pos_attr_thr, 
    neg_attr_thr, pos_rel_thr, neg_rel_thr):
    rule_list = [] # All rules
    n_cluster = len(data_["cls"].drop_duplicates()) # N clusters
    attrs = data_.columns[:-1] # Name of the columns

    for C_i in range(n_cluster):
        #print(C_i)
        rule_i = []
        data_cluster = data_[data_["cls"] == C_i]
        
        # Effective attributes
        attr_filters = extract_attributes_filters(data_cluster, attrs, data_, 
            pos_attr_thr, neg_attr_thr)    
        rule_i.append(attr_filters)        

        # Relations
        attr_relation = extract_relations(data_cluster, attrs, data_, 
            pos_rel_thr, neg_rel_thr)
        rule_i.append(attr_relation)
        #print(rule_i)

        rule_list.append([C_i, rule_i])    

    return rule_list

In [7]:
df_test = df_pos_entries[df_pos_entries.columns[1:]]
df_test.columns

Index(['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME',
       'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE', 'cls'],
      dtype='object')

In [27]:
pos_attr_thr = 0.3
neg_attr_thr = 0.2
pos_rel_thr = 0.3
neg_rel_thr = 0.2
test = rule_inference(df_test, pos_attr_thr, neg_attr_thr, pos_rel_thr, neg_attr_thr)
test

[[0, [[['ROLE_ROLLUP_1', -117961]], None]],
 [1,
  [[['ROLE_ROLLUP_2', 118052],
    ['ROLE_TITLE', 118784],
    ['ROLE_FAMILY_DESC', -117906],
    ['ROLE_CODE', 118786]],
   None]],
 [2, [[['ROLE_ROLLUP_2', 118300]], None]],
 [3, [[['ROLE_ROLLUP_2', 118386]], None]],
 [4, [[['ROLE_ROLLUP_2', 118327]], None]],
 [5,
  [[['MGR_ID', 141],
    ['ROLE_ROLLUP_1', 117961],
    ['ROLE_ROLLUP_2', 118225],
    ['ROLE_DEPTNAME', 122273],
    ['ROLE_TITLE', 118321],
    ['ROLE_FAMILY', 290919],
    ['ROLE_CODE', 118322]],
   None]],
 [6,
  [[['ROLE_ROLLUP_2', 118446], ['ROLE_TITLE', 118259], ['ROLE_CODE', 118261]],
   None]],
 [7,
  [[['ROLE_ROLLUP_2', 118343],
    ['ROLE_FAMILY', 118424],
    ['ROLE_FAMILY', -290919]],
   None]],
 [8,
  [[['ROLE_TITLE', 117905],
    ['ROLE_FAMILY_DESC', 117906],
    ['ROLE_FAMILY', 290919],
    ['ROLE_CODE', 117908]],
   None]],
 [9, [[['ROLE_ROLLUP_1', 117961], ['ROLE_ROLLUP_2', 117962]], None]],
 [10,
  [[['ROLE_ROLLUP_1', 117961],
    ['ROLE_ROLLUP_2', 118343],

In [29]:
only_rules = []
for rule in test:
    only_rules.append(rule[1][0])
only_rules

[[['ROLE_ROLLUP_1', -117961]],
 [['ROLE_ROLLUP_2', 118052],
  ['ROLE_TITLE', 118784],
  ['ROLE_FAMILY_DESC', -117906],
  ['ROLE_CODE', 118786]],
 [['ROLE_ROLLUP_2', 118300]],
 [['ROLE_ROLLUP_2', 118386]],
 [['ROLE_ROLLUP_2', 118327]],
 [['MGR_ID', 141],
  ['ROLE_ROLLUP_1', 117961],
  ['ROLE_ROLLUP_2', 118225],
  ['ROLE_DEPTNAME', 122273],
  ['ROLE_TITLE', 118321],
  ['ROLE_FAMILY', 290919],
  ['ROLE_CODE', 118322]],
 [['ROLE_ROLLUP_2', 118446], ['ROLE_TITLE', 118259], ['ROLE_CODE', 118261]],
 [['ROLE_ROLLUP_2', 118343],
  ['ROLE_FAMILY', 118424],
  ['ROLE_FAMILY', -290919]],
 [['ROLE_TITLE', 117905],
  ['ROLE_FAMILY_DESC', 117906],
  ['ROLE_FAMILY', 290919],
  ['ROLE_CODE', 117908]],
 [['ROLE_ROLLUP_1', 117961], ['ROLE_ROLLUP_2', 117962]],
 [['ROLE_ROLLUP_1', 117961],
  ['ROLE_ROLLUP_2', 118343],
  ['ROLE_TITLE', 118321],
  ['ROLE_FAMILY_DESC', 117906],
  ['ROLE_FAMILY', 290919],
  ['ROLE_CODE', 118322]],
 [['ROLE_ROLLUP_2', 118300],
  ['ROLE_TITLE', 118784],
  ['ROLE_FAMILY_DESC', 117

In [30]:
# Compute how many rules has a lenght equal to 1.
for idx, rule in enumerate(only_rules):
    if len(rule) == 1:
        print(idx)

0
2
3
4


In [14]:
del only_rules[0]
del only_rules[2]
del only_rules[3]
del only_rules[4]

In [31]:
false_neg  = []
for i,row in df_pos_entries.iterrows():
    
    # Evaluación
    denies_count = 0    
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True
        
        #for idx_r, attr_val in enumerate(rule):
        for attr_val in rule:            
            if attr_val[1] < 0:
                if row[attr_val[0]] != attr_val[1]*-1:
                    res = False
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1
    
    if denies_count == len(only_rules):
        false_neg.append(row)
        #print("FP-2")

In [32]:
FN = len(false_neg)
print("Tasa FN: {:.2f}".format((FN/ len(df_pos_entries))*100))
print("FN: ", FN, " de ", len(df_pos_entries))

Tasa FN: 32.76
FN:  10114  de  30872


In [33]:
false_pos  = []
for i,row in df_neg_entries.iterrows():
    # Evaluación
    denies_count = 0
    temp_rules_n = 0
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True                        
        for attr_val in rule:
            if attr_val[1] < 0:
                if row[attr_val[0]] != attr_val[1]*-1:
                    res = False
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1                                
    #print("XXX-", denies_count, temp_rules_n, res)
    if denies_count < len(only_rules):
        false_pos.append(row)
        #print("FP-2")    
    #else:
    #    print("ENtra PAPA")
    

In [34]:
FP = len(false_pos)
print("Tasa FP: {:.2f}".format((FP/ len(df_neg_entries))*100))
print("FN: ", FP, " de ", len(df_neg_entries))

Tasa FP: 59.41
FN:  1127  de  1897


In [35]:

TP = len(df_pos_entries) - FN
#TP = 50 - FN
TN = len(df_neg_entries) - FP
#TN = 50 - FP

precision = TP / (TP + FP)

recall = TP / (TP + FN)

fscore = 2*(precision*recall)/(precision+recall)

print("FN:", FN, " - {:.2f}".format((FN/len(df_pos_entries))*100))
#print("FN:", FN, " - {:.2f}".format((FN/50)*100))
print("FP:", FP, " - {:.2f}".format((FP/len(df_neg_entries))*100))
#print("FP:", FP, " - {:.2f}".format((FP/50)*100))
print("Precision:", precision)
print("Recall:", recall)
print("F-score", fscore)

def compute_wsc(policy):
    return sum([len(rule) for rule in policy])

print("# Rules:", len(only_rules))
print("WSC:", compute_wsc(only_rules))

FN: 10114  - 32.76
FP: 1127  - 59.41
Precision: 0.9485035412382911
Recall: 0.6723892200051826
F-score 0.7869287487916296
# Rules: 15
WSC: 52


## Policy Enhancement
1. Rule Pruning
2. Policy Refinement

In [41]:
false_neg = pd.DataFrame(false_neg)

In [42]:
###Select the number of clusters###
num_clusters = 5

#DO NOT CHANGE THIS CODE
# seed = 29

#Compute centroids and labels
# num_init = 5
centroids = []
kmodes_huang = kmodes.KModes(n_clusters=num_clusters, init='Huang', verbose=0)
cluster_labels = kmodes_huang.fit_predict(false_neg)
centroids = kmodes_huang.cluster_centroids_

print('Ready!')    

Ready!


In [43]:
false_neg["cls"] = cluster_labels

In [44]:
df_test_2 = false_neg[false_neg.columns[1:]]

In [45]:
pos_attr_thr = 0.3
neg_attr_thr = 0.2
pos_rel_thr = 0.3
neg_rel_thr = 0.2
test = rule_inference(df_test_2, pos_attr_thr, neg_attr_thr, pos_rel_thr, neg_attr_thr)
test

[[0, [[['ROLE_FAMILY', -19721]], None]],
 [1,
  [[['ROLE_TITLE', 118568], ['ROLE_FAMILY', 19721], ['ROLE_CODE', 118570]],
   None]],
 [2,
  [[['ROLE_DEPTNAME', 117878],
    ['ROLE_TITLE', 117879],
    ['ROLE_FAMILY_DESC', 117879],
    ['ROLE_FAMILY', 19721],
    ['ROLE_CODE', 117880]],
   None]],
 [3,
  [[['ROLE_ROLLUP_1', 91261],
    ['ROLE_ROLLUP_2', 118026],
    ['ROLE_TITLE', 118321],
    ['ROLE_FAMILY_DESC', 117906],
    ['ROLE_FAMILY', 290919],
    ['ROLE_CODE', 118322]],
   None]],
 [4,
  [[['ROLE_DEPTNAME', 117920],
    ['ROLE_TITLE', 117879],
    ['ROLE_FAMILY_DESC', 117886],
    ['ROLE_FAMILY', 19721],
    ['ROLE_CODE', 117880]],
   None]]]

In [46]:
#only_rules = []
for rule in test:
    only_rules.append(rule[1][0])
len(only_rules)

20

In [47]:
# Compute how many rules has a lenght equal to 1.
for idx, rule in enumerate(only_rules):
    if len(rule) == 1:
        print(idx)

0
2
3
4
15


In [48]:
false_neg  = []
for i,row in df_pos_entries.iterrows():
    
    # Evaluación
    denies_count = 0    
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True
        
        #for idx_r, attr_val in enumerate(rule):
        for attr_val in rule:            
            if attr_val[1] < 0:
                if row[attr_val[0]] == attr_val[1]*-1:
                    res = False
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1
    
    if denies_count == len(only_rules):
        false_neg.append(row)
        #print("FP-2")

FN = len(false_neg)
print("Tasa FN: {:.2f}".format((FN/ len(df_pos_entries))*100))
print("FN: ", FN, " de ", len(df_pos_entries))

Tasa FN: 24.60
FN:  7595  de  30872


In [49]:
false_pos  = []
for i,row in df_neg_entries.iterrows():
    # Evaluación
    denies_count = 0
    temp_rules_n = 0
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True                        
        for attr_val in rule:
            if attr_val[1] < 0:
                if row[attr_val[0]] == attr_val[1]*-1:
                    res = False
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1                                
    #print("XXX-", denies_count, temp_rules_n, res)
    if denies_count < len(only_rules):
        false_pos.append(row)
        #print("FP-2")    
    #else:
    #    print("ENtra PAPA")
FP = len(false_pos)
print("Tasa FP: {:.2f}".format((FP/ len(df_neg_entries))*100))
print("FN: ", FP, " de ", len(df_neg_entries))

Tasa FP: 72.54
FN:  1376  de  1897


In [50]:
TP = len(df_pos_entries) - FN
#TP = 50 - FN
TN = len(df_neg_entries) - FP
#TN = 50 - FP

precision = TP / (TP + FP)

recall = TP / (TP + FN)

fscore = 2*(precision*recall)/(precision+recall)

print("FN:", FN, " - {:.2f}".format((FN/len(df_pos_entries))*100))
#print("FN:", FN, " - {:.2f}".format((FN/50)*100))
print("FP:", FP, " - {:.2f}".format((FP/len(df_neg_entries))*100))
#print("FP:", FP, " - {:.2f}".format((FP/50)*100))
print("Precision:", precision)
print("Recall:", recall)
print("F-score", fscore)

def compute_wsc(policy):
    return sum([len(rule) for rule in policy])

print("# Rules:", len(only_rules))
print("WSC:", compute_wsc(only_rules))

FN: 7595  - 24.60
FP: 1376  - 72.54
Precision: 0.9441852918508904
Recall: 0.7539841927960611
F-score 0.8384331382260243
# Rules: 20
WSC: 72
